In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
import glob
import os
import metpy
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
from datetime import datetime
import xarray as xr
from pint import UnitRegistry
ureg = UnitRegistry()
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway
import statsmodels.api as sm
from scipy.stats import t
from scipy.stats import pearsonr

In [2]:


# Load datasets
df1 = pd.read_csv("C:\\YEAR1\\NMVOC\\Data\\processed_c3h8.csv", parse_dates=['datetime'])
df2 = pd.read_csv("C:\\YEAR1\\NMVOC\\Data\\processed_CH4.csv", parse_dates=['datetime'])
df3 = pd.read_csv("C:\\YEAR1\\NMVOC\\Data\\processed_CO.csv", parse_dates=['datetime'])
df4 = pd.read_csv("C:\\YEAR1\\NMVOC\Data\\processed_O3.csv", parse_dates=['datetime'])

# Merge all datasets on 'datetime' using an inner join
merged_df = df1.merge(df2, on='datetime', how='inner') \
               .merge(df3, on='datetime', how='inner') \
               .merge(df4, on='datetime', how='inner')

# Display result
print(merged_df)


<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\EnricoMancinelli\AppData\Local\Temp\ipykernel_5864\3373939585.py:5: SyntaxWarning: invalid escape sequence '\D'
  df4 = pd.read_csv("C:\\YEAR1\\NMVOC\Data\\processed_O3.csv", parse_dates=['datetime'])


                 datetime  C3H8_ppb  CH4_ppb    CO_ppb  O3_ppb
0     2011-06-17 11:00:00   0.14212  1844.85   93.1660    50.5
1     2011-06-17 13:00:00   0.15840  1856.25  102.3575    57.1
2     2011-06-17 17:00:00   0.08721  1835.98   93.6670    54.6
3     2011-06-17 23:00:00   0.05857  1839.14   90.8355    54.9
4     2011-06-18 01:00:00   0.06136  1839.20   92.8540    57.0
...                   ...       ...      ...       ...     ...
27335 2023-12-30 17:00:00   0.07839  1992.76  122.0020    39.0
27336 2023-12-30 19:00:00   0.09994  1988.35  117.0450    42.1
27337 2023-12-30 20:00:00   0.11000  1984.59  114.0310    42.3
27338 2023-12-30 22:00:00   0.24459  1981.67  113.9700    42.2
27339 2023-12-30 23:00:00   0.26068  1980.46  112.5030    42.4

[27340 rows x 5 columns]


In [3]:
merged_df.corr().style.background_gradient(cmap='coolwarm')

,datetime,C3H8_ppb,CH4_ppb,CO_ppb,O3_ppb
datetime,1.000000,-0.005806,0.794266,0.079071,-0.038210
C3H8_ppb,-0.005806,1.000000,0.164602,0.556258,-0.341281
CH4_ppb,0.794266,0.164602,1.000000,0.431503,0.094697
CO_ppb,0.079071,0.556258,0.431503,1.000000,0.070354
O3_ppb,-0.038210,-0.341281,0.094697,0.070354,1.000000


In [4]:


# Extract month and season
merged_df['month'] = merged_df['datetime'].dt.month

# Define seasons: (Winter: Dec-Feb, Spring: Mar-May, Summer: Jun-Aug, Fall: Sep-Nov)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

merged_df['season'] = merged_df['month'].apply(get_season)




In [5]:


# Drop non-numeric columns (datetime, month, and season) before correlation
numeric_df = merged_df.drop(columns=['datetime', 'month', 'season'])

# Group by month and compute correlation
corr_per_month = numeric_df.groupby(merged_df['month']).corr()
corr_per_month = corr_per_month.round(3)
# Group by season and compute correlation
corr_per_season = numeric_df.groupby(merged_df['season']).corr()
corr_per_season = corr_per_season.round(3)

In [6]:
save_dir="C:\\YEAR1\\NMVOC\\figures\\"
# Save final  DataFrame
output_file  = os.path.join(save_dir, "corr_month.csv")
corr_per_month.to_csv(output_file, index=True)
output_file1  = os.path.join(save_dir, "corr_season.csv")
corr_per_season.to_csv(output_file1, index=True)

# Display results
print("Correlation per Month:\n", corr_per_month)
print("\nCorrelation per Season:\n", corr_per_season)

Correlation per Month:
                 C3H8_ppb  CH4_ppb  CO_ppb  O3_ppb
month                                            
1     C3H8_ppb     1.000    0.202   0.570  -0.233
      CH4_ppb      0.202    1.000   0.624   0.135
      CO_ppb       0.570    0.624   1.000  -0.165
      O3_ppb      -0.233    0.135  -0.165   1.000
2     C3H8_ppb     1.000    0.310   0.495   0.045
      CH4_ppb      0.310    1.000   0.207  -0.013
      CO_ppb       0.495    0.207   1.000  -0.178
      O3_ppb       0.045   -0.013  -0.178   1.000
3     C3H8_ppb     1.000    0.359   0.411  -0.019
      CH4_ppb      0.359    1.000   0.387  -0.009
      CO_ppb       0.411    0.387   1.000   0.204
      O3_ppb      -0.019   -0.009   0.204   1.000
4     C3H8_ppb     1.000    0.473   0.563   0.024
      CH4_ppb      0.473    1.000   0.374   0.007
      CO_ppb       0.563    0.374   1.000   0.239
      O3_ppb       0.024    0.007   0.239   1.000
5     C3H8_ppb     1.000    0.355   0.457   0.044
      CH4_ppb      0.355  

In [7]:
corr_per_month = numeric_df.groupby(merged_df['month'])
rho_m = corr_per_month.corr()
pval = corr_per_month.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho_m.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
rho_m.round(3).astype(str) + p

C:\Users\EnricoMancinelli\AppData\Local\Temp\ipykernel_5864\1222399242.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))


C3H8_ppb    CH4_ppb     CO_ppb     O3_ppb
month                                                     
1     C3H8_ppb     1.0***   0.202***    0.57***  -0.233***
      CH4_ppb    0.202***     1.0***   0.624***   0.135***
      CO_ppb      0.57***   0.624***     1.0***  -0.165***
      O3_ppb    -0.233***   0.135***  -0.165***     1.0***
2     C3H8_ppb        1.0    0.31***   0.495***     0.045*
      CH4_ppb     0.31***        1.0   0.207***     -0.013
      CO_ppb     0.495***   0.207***        1.0  -0.178***
      O3_ppb       0.045*     -0.013  -0.178***        1.0
3     C3H8_ppb        1.0   0.359***   0.411***     -0.019
      CH4_ppb    0.359***        1.0   0.387***     -0.009
      CO_ppb     0.411***   0.387***        1.0   0.204***
      O3_ppb       -0.019     -0.009   0.204***        1.0
4     C3H8_ppb        1.0   0.473***   0.563***      0.024
      CH4_ppb    0.473***        1.0   0.374***      0.007
      CO_ppb     0.563***   0.374***        1.0   0.239***
      O3_ppb        0.024      0.007   0.239***        1.0
5     C3H8_ppb        1.0   0.355***   0.457***     0.044*
      CH4_ppb    0.355***        1.0   0.377***  -0.196***
      CO_ppb     0.457***   0.377***        1.0   0.231***
      O3_ppb       0.044*  -0.196***   0.231***        1.0
6     C3H8_ppb        1.0   0.496***   0.547***   0.321***
      CH4_ppb    0.496***        1.0   0.624***   0.343***
      CO_ppb     0.547***   0.624***        1.0   0.473***
      O3_ppb     0.321***   0.343***   0.473***        1.0
7     C3H8_ppb        1.0   0.395***    0.39***   0.295***
      CH4_ppb    0.395***        1.0    0.42***   0.132***
      CO_ppb      0.39***    0.42***        1.0   0.391***
      O3_ppb     0.295***   0.132***   0.391***        1.0
8     C3H8_ppb        1.0   0.262***   0.415***   0.431***
      CH4_ppb    0.262***        1.0   0.505***   0.208***
      CO_ppb     0.415***   0.505***        1.0    0.55***
      O3_ppb     0.431***   0.208***    0.55***        1.0
9     C3H8_ppb        1.0   0.215***   0.373***   0.228***
      CH4_ppb    0.215***        1.0   0.515***   0.086***
      CO_ppb     0.373***   0.515***        1.0   0.445***
      O3_ppb     0.228***   0.086***   0.445***        1.0
10    C3H8_ppb        1.0   0.215***   0.468***     -0.015
      CH4_ppb    0.215***        1.0   0.552***     -0.024
      CO_ppb     0.468***   0.552***        1.0   0.263***
      O3_ppb       -0.015     -0.024   0.263***        1.0
11    C3H8_ppb        1.0   0.166***   0.631***  -0.281***
      CH4_ppb    0.166***        1.0    0.51***  -0.222***
      CO_ppb     0.631***    0.51***        1.0  -0.425***
      O3_ppb    -0.281***  -0.222***  -0.425***        1.0
12    C3H8_ppb        1.0   0.233***   0.626***  -0.476***
      CH4_ppb    0.233***        1.0    0.42***   -0.32***
      CO_ppb     0.626***    0.42***        1.0  -0.649***
      O3_ppb    -0.476***   -0.32***  -0.649***        1.0

In [8]:
corr_per_season = numeric_df.groupby(merged_df['season'])
rho_s = corr_per_season.corr()
pval = corr_per_season.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho_s.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
rho_s.round(3).astype(str) + p

C:\Users\EnricoMancinelli\AppData\Local\Temp\ipykernel_5864\2847383662.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))


C3H8_ppb    CH4_ppb     CO_ppb     O3_ppb
season                                                     
Fall   C3H8_ppb     1.0***   0.182***   0.449***  -0.254***
       CH4_ppb    0.182***     1.0***   0.533***     -0.022
       CO_ppb     0.449***   0.533***     1.0***   0.139***
       O3_ppb    -0.254***     -0.022   0.139***     1.0***
Spring C3H8_ppb        1.0   0.246***   0.609***   -0.12***
       CH4_ppb    0.246***        1.0   0.305***   -0.034**
       CO_ppb     0.609***   0.305***        1.0   0.107***
       O3_ppb     -0.12***   -0.034**   0.107***        1.0
Summer C3H8_ppb        1.0   0.384***   0.442***   0.343***
       CH4_ppb    0.384***        1.0   0.482***   0.218***
       CO_ppb     0.442***   0.482***        1.0   0.472***
       O3_ppb     0.343***   0.218***   0.472***        1.0
Winter C3H8_ppb        1.0   0.227***    0.58***  -0.159***
       CH4_ppb    0.227***        1.0   0.385***  -0.045***
       CO_ppb      0.58***   0.385***        1.0  -0.247***
       O3_ppb    -0.159***  -0.045***  -0.247***        1.0

In [9]:
save_dir="C:\\YEAR1\\NMVOC\\figures\\"
# Save final  DataFrame
output_file  = os.path.join(save_dir, "corr_month.csv")
rho_m.to_csv(output_file, index=True)
output_file1  = os.path.join(save_dir, "corr_season.csv")
rho_s.to_csv(output_file1, index=True)